In [1]:
pip install langchain langchain-community langchain-google-genai faiss-cpu


Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
if "USER_AGENT" not in os.environ:
    os.environ["USER_AGENT"] = "ask-your-data/1.0"

In [3]:
from langchain.document_loaders import TextLoader, PyPDFLoader, WebBaseLoader

# Ask user for data source type
data_source = input("Enter data source type ('file' or 'webpage'): ").strip().lower()

if data_source == "webpage":
    url = input("Enter the webpage URL: ").strip()
    loader = WebBaseLoader(url)
    documents = loader.load()
else:
    file_path = input("Enter the file path (e.g., data/myfile.txt or data/myfile.pdf): ").strip()
    # Choose loader based on file extension
    if file_path.lower().endswith(".pdf"):
        loader = PyPDFLoader(file_path)
    else:
        loader = TextLoader(file_path)
    documents = loader.load()

In [4]:
# Split the documents into chunks
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(separator="\n", chunk_size=200, chunk_overlap=20)
split_docs = splitter.split_documents(documents)

print(f"Loaded {len(documents)} documents")
print(f"Split into {len(split_docs)} chunks")
if len(split_docs) == 0:
    print("No chunks to embed! Check your file and splitter settings.")


Created a chunk of size 576, which is longer than the specified 200
Created a chunk of size 768, which is longer than the specified 200
Created a chunk of size 404, which is longer than the specified 200
Created a chunk of size 307, which is longer than the specified 200
Created a chunk of size 322, which is longer than the specified 200
Created a chunk of size 510, which is longer than the specified 200
Created a chunk of size 317, which is longer than the specified 200
Created a chunk of size 505, which is longer than the specified 200
Created a chunk of size 484, which is longer than the specified 200
Created a chunk of size 437, which is longer than the specified 200
Created a chunk of size 272, which is longer than the specified 200
Created a chunk of size 330, which is longer than the specified 200


Loaded 1 documents
Split into 26 chunks


In [8]:
if "GOOGLE_API_KEY" not in os.environ:
    raise ValueError("Please set the GOOGLE_API_KEY environment variable before running the notebook.")

ValueError: Please set the GOOGLE_API_KEY environment variable before running the notebook.

In [ ]:
# Use Gemini embeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
db = FAISS.from_documents(split_docs, embeddings)

In [ ]:
#get from user
prompt = input()

In [ ]:
results = db.similarity_search(prompt, k=3)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
prompt_template = ChatPromptTemplate.from_template(
    """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:""")
qa_chain = create_stuff_documents_chain(llm, prompt_template)

response = qa_chain.invoke({"context": results, "question": prompt})
print(response)

The provided text does not state what IIT Jodhpur was initially called. It only mentions that it was sanctioned as an IIT for Rajasthan. Thanks for asking!
